In [35]:
import logging
import os
import platform

import numpy as np
import pybullet as p
import yaml

import igibson
from igibson.action_primitives.action_primitive_set_base import ActionPrimitiveError
from igibson.action_primitives.starter_semantic_action_primitives import StarterSemanticActionPrimitives
from igibson.objects.articulated_object import URDFObject
from igibson.robots.behavior_robot import BehaviorRobot
from igibson.robots.fetch import Fetch
from igibson.scenes.igibson_indoor_scene import InteractiveIndoorScene
from igibson.simulator import Simulator
from igibson.utils.assets_utils import get_ig_avg_category_specs, get_ig_model_path
from igibson.utils.utils import parse_config

In [36]:
def execute_controller(ctrl_gen, robot, s):
    for action in ctrl_gen:
        robot.apply_action(action)
        s.step()


def go_to_sink_and_toggle(s, robot, controller: StarterSemanticActionPrimitives):
    """Go to the sink object in the scene and toggle it on."""
    for i in range(20):
        try:
            sink = s.scene.objects_by_category["sink"][1]
            print("Trying to NAVIGATE_TO sink.")
            execute_controller(controller._navigate_to_obj(sink), robot, s)
            print("NAVIGATE_TO sink succeeded!")
            print("Trying to TOGGLE_ON the sink.")
            execute_controller(controller.toggle_on(sink), robot, s)
            print("TOGGLE_ON the sink succeeded!")
        except ActionPrimitiveError:
            print("Attempt {} to navigate and toggle on the sink failed. Retry until 20.".format(i + 1))
            continue

        return


def grasp_tray(s, robot, controller: StarterSemanticActionPrimitives):
    """Grasp the tray that's on the floor of the room."""
    for i in range(20):
        try:
            print("Trying to GRASP tray.")
            tray = s.scene.objects_by_category["tray"][0]
            execute_controller(controller.grasp(tray), robot, s)
            print("GRASP the tray succeeded!")
        except ActionPrimitiveError:
            print("Attempt {} to grasp the tray failed. Retry until 20.".format(i + 1))
            continue

        return


def put_on_table(s, robot, controller: StarterSemanticActionPrimitives):
    """Place the currently-held object on top of the coffee table."""
    for i in range(20):
        try:
            print("Trying to PLACE_ON_TOP the held object on coffee table.")
            table = s.scene.objects_by_category["coffee_table"][0]
            execute_controller(controller.place_on_top(table), robot, s)
            print("PLACE_ON_TOP succeeded!")
        except ActionPrimitiveError:
            print("Attempt {} to place the held object failed. Retry until 20.".format(i + 1))
            continue

        return


def open_and_close_fridge(s, robot, controller: StarterSemanticActionPrimitives):
    """Demonstrate opening and closing the fridge."""
    for i in range(20):
        try:
            fridge = s.scene.objects_by_category["fridge"][0]
            print("Trying to OPEN the fridge.")
            execute_controller(controller.open(fridge), robot, s)
            print("OPEN the fridge succeeded!")
            print("Trying to CLOSE the fridge.")
            execute_controller(controller.close(fridge), robot, s)
            print("CLOSE the fridge succeeded!")
        except ActionPrimitiveError:
            print("Attempt {} to open and close the fridge failed. Retry until 20.".format(i + 1))
            continue

        return


def open_and_close_door(s, robot, controller: StarterSemanticActionPrimitives):
    """Demonstrate opening and closing the bathroom door."""
    for i in range(20):
        try:
            door = (set(s.scene.objects_by_category["door"]) & set(s.scene.objects_by_room["bathroom_0"])).pop()
            print("Trying to OPEN the door.")
            execute_controller(controller.open(door), robot, s)
            print("Trying to CLOSE the door.")
            execute_controller(controller.close(door), robot, s)
            print("CLOSE the door succeeded!")
        except ActionPrimitiveError:
            print("Attempt {} to open and close the door failed. Retry until 20.".format(i + 1))
            continue

        return


def open_and_close_cabinet(s, robot, controller: StarterSemanticActionPrimitives):
    """Demonstrate opening and closing a drawer unit."""
    for i in range(20):
        try:
            cabinet = s.scene.objects_by_category["bottom_cabinet"][2]
            print("Trying to OPEN the cabinet.")
            execute_controller(controller.open(cabinet), robot, s)
            print("Trying to CLOSE the cabinet.")
            execute_controller(controller.close(cabinet), robot, s)
            print("CLOSE the cabinet succeeded!")
        except ActionPrimitiveError:
            print("Attempt {} to open and close the cabinet failed. Retry until 20.".format(i + 1))
            continue

        return

In [86]:
logging.basicConfig(level=logging.INFO)

s = Simulator(
    mode="headless",
    image_width=512,
    image_height=512,
    device_idx=0,
    use_pb_gui=False,
)
scene = InteractiveIndoorScene(
    "Rs_int", load_object_categories=["walls", "floors", "bottom_cabinet", "door", "sink", "coffee_table", "fridge"]
)
s.import_scene(scene)

# Create a custom tray object for the grasping test.
model_path = get_ig_model_path("tray", "tray_000")
model_filename = os.path.join(model_path, "tray_000.urdf")
avg_category_spec = get_ig_avg_category_specs()
tray = URDFObject(
    filename=model_filename,
    category="tray",
    name="tray",
    avg_obj_dims=avg_category_spec.get("tray"),
    fit_avg_dim_volume=True,
    model_path=model_path,
)
s.import_object(tray)
tray.set_position_orientation([0, 1, 0.3], p.getQuaternionFromEuler([0, np.pi / 2, 0]))

# Load the robot and place it in the scene.
#config = parse_config(os.path.join(igibson.configs_path, "behavior_robot_mp_behavior_task.yaml"))
config = parse_config(os.path.join(igibson.configs_path, "fetch_motion_planning.yaml"))

#config["robot"]["show_visual_head"] = True
robot = Fetch(**config["robot"])
s.import_robot(robot)
robot.set_position_orientation([0, 0, 1], p.getQuaternionFromEuler([0, 0, -np.pi/4])) # positive offset on the third dim -> turn left, negavite -> turn right
robot.apply_action(
    np.zeros(
        robot.action_dim,
    )
)

# Run some steps to let physics settle.
for _ in range(300):
    s.step()

INFO:igibson.render.mesh_renderer.get_available_devices:Device 0 is available for rendering
INFO:igibson.render.mesh_renderer.get_available_devices:Command '['/home/nicola/python_projects/iGibson/igibson/render/mesh_renderer/build/test_device', '1']' returned non-zero exit status 1.
INFO:igibson.render.mesh_renderer.get_available_devices:Device 1 is not available for rendering
INFO:igibson.scenes.scene_base:Loading scene...
INFO:igibson.scenes.scene_base:Scene loaded!


In [87]:
robot_pos, robot_orientation = robot.get_position_orientation()

In [88]:
robot_orientation

array([ 0.00453328,  0.01126892, -0.38878708,  0.92124756])

In [92]:
import matplotlib.pyplot as plt
import os
from PIL import Image

def get_view_dir(angle_deg):
    """
    Given angle in degrees, return the view direction in the xy plane corresponding to the angle. 
    Consider that: 
    angle_deg=0 -> np.array([1,0,0]) 
    angle_deg=90 -> np.array([0,1,0])
    """
    import math
    angle_rad = angle_deg * math.pi / 180 
    x = math.cos(angle_rad)
    y = math.sin(angle_rad)
    return np.array([x,y,0])

angles = [0, 45, 90, 135, 180, 225, 270, 315, 360]
for angle in angles:
    # from examples/renderer/mesh_renderer_gpu_example
    camera_pose = robot_pos + q.rotate(np.array([0.1, 0.1, 1])) #+ np.array([0.5, 0.5, 0.5])
    view_direction = q.rotate(get_view_dir(angle))
    print(f"Angle: {angle} - View direction: {view_direction}")
    
    s.renderer.set_camera(camera_pose, camera_pose + view_direction, q.rotate([0, 0, 1]))
    s.renderer.set_fov(120)
    
    frame = s.renderer.render(modes=("rgb"))[0]
    rgb_image = (frame[..., :3] * 255).astype(np.uint8) 
    
    # Ensure the directory exists
    os.makedirs("./images/rendering_attempts", exist_ok=True)
    
    # Save using PIL
    image = Image.fromarray(rgb_image)
    #image.show()
    image.save(f"./images/rendering_attempts/img_{angle}_deg.jpg", "JPEG")
    
    print("Image saved at "+f"./images/rendering_attempts/img_{angle}_deg.jpg")


Angle: 0 - View direction: [ 0.69743523 -0.71623613 -0.0242879 ]
Image saved at ./images/img_0_deg.jpg
Angle: 45 - View direction: [ 0.9997611  -0.01314371 -0.01746396]
Image saved at ./images/img_45_deg.jpg
Angle: 90 - View direction: [ 7.1644047e-01  6.9764811e-01 -4.0987053e-04]
Image saved at ./images/img_90_deg.jpg
Angle: 135 - View direction: [0.01343873 0.99976713 0.01688432]
Image saved at ./images/img_135_deg.jpg
Angle: 180 - View direction: [-0.69743523  0.71623613  0.0242879 ]
Image saved at ./images/img_180_deg.jpg
Angle: 225 - View direction: [-0.9997611   0.01314371  0.01746396]
Image saved at ./images/img_225_deg.jpg
Angle: 270 - View direction: [-7.1644047e-01 -6.9764811e-01  4.0987053e-04]
Image saved at ./images/img_270_deg.jpg
Angle: 315 - View direction: [-0.01343873 -0.99976713 -0.01688432]
Image saved at ./images/img_315_deg.jpg
Angle: 360 - View direction: [ 0.69743523 -0.71623613 -0.0242879 ]
Image saved at ./images/img_360_deg.jpg


In [95]:
# Can we get the robot camera PoV?
import pyquaternion  

robot_pos, robot_orientation = robot.get_position_orientation()

# Convert quaternion to rotation matrix - takes w,x,y,z in input, but robot orientation is given as x,y,z,w !!!
q = pyquaternion.Quaternion(x=robot_orientation[0], 
                            y=robot_orientation[1], 
                            z=robot_orientation[2], 
                            w=robot_orientation[3])

forward_downward_direction = q.rotate(np.array([1, 0, -0.25]))  # Default forward vector (x-axis)
up_direction = q.rotate(np.array([0, 0, 1]))  # Default up vector (z-axis)

# Set the camera at the robot's head level (optional: raise it slightly)
camera_pose = robot_pos + q.rotate(np.array([0.1, 0.1, 1])) # Slightly above the robot's center

# Set the camera in the renderer
s.renderer.set_camera(camera_pose, camera_pose + forward_downward_direction, up_direction)
frame = s.renderer.render(modes=("rgb"))[0]
rgb_image = (frame[..., :3] * 255).astype(np.uint8) 

# Ensure the directory exists
os.makedirs("./images/rendering_attempts", exist_ok=True)

# Save using PIL
image = Image.fromarray(rgb_image)
#image.show()
image.save(f"./images/rendering_attempts/img_1st_person.jpg", "JPEG")